In [1]:
import json
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
from copy import copy
import sys
sys.path.append('/home/ndsviriden/MinMax94/src/utils') 
from interpolation import interpolate_mmx, create_patterns
from constants import mmcc_forecast_columns 
from converters import convert_rp5_to_mmx, convert_raw_to_mmx, convert_mmx_to_mmcc_forecast, convert_raw_to_mmx, \
                convert_mmx_to_mmcc_rwis
from loaders import load_rp5_stations, load_mm94_stations
from geographical import find_nearest_wmo_station
from mmcc_input import get_mmcc_input_json, get_mmcc_prediction


## Choose MM94 station

In [2]:
mm94_station_id = 115

raw = load_mm94_stations(mm94_station_id)
mmx_rwis = convert_raw_to_mmx(raw)
mmx_rwis_interpolated = interpolate_mmx(mmx_rwis)
mmx_rwis_interpolated = create_patterns(mmx_rwis_interpolated)
mmcc_rwis = convert_mmx_to_mmcc_rwis(mmx_rwis_interpolated)
mmcc_rwis = mmcc_rwis.set_index("date_time_utc")

In [3]:
wmo_station_id = find_nearest_wmo_station(mm94_station_id, verbose=True)

rp5 = load_rp5_stations(wmo_station_id)
mmx_forecast = convert_rp5_to_mmx(rp5)
mmx_forecast_interpolated = interpolate_mmx(mmx_forecast)
mmcc_forecast = convert_mmx_to_mmcc_forecast(mmx_forecast_interpolated)
mmcc_forecast = mmcc_forecast.set_index("date_time_utc")

distance([ 54.646  55.546], [ 54.75  55.98]) = 30.28 km


In [ ]:
start = pd.Timestamp(2014, 1, 1)
end = pd.Timestamp(2015, 10, 1, 12)
#mmcc_rwis_part = mmcc_rwis[(mmcc_rwis.index <= end) & (mmcc_rwis.index >= start)]
mmcc_rwis_part = copy(mmcc_rwis)

## Get station configuration

In [ ]:
import requests
from time import sleep
import simplejson

prediction_list = []

for time_record in tqdm_notebook(mmcc_rwis_part.index):
    data = get_mmcc_input_json(mm94_station_id, mmcc_rwis, mmcc_forecast, time_record)
    
    prediction = get_mmcc_prediction(data)
    
    if prediction['roadcast']:
        data = {'date_time_utc': str(time_record), 'roadcast': prediction['roadcast']}
        prediction_list.append(data)
    
    else:
        print("{0} fails".format(time_record))
        print("-----------------------------")
        

to_save = {'station_id': mm94_station_id, 'meteo_data': prediction_list}

with open('/mnt/HARD/MinMax94/data/JSON/{0}_metro.txt'.format(mm94_station_id), "w") as file:
    json.dump(to_save, file, indent=2)

b'{\n  "error": "rwis_data should contain at least two records with valid t_road."\n}\n'
2012-09-17 15:00:00 fails
-----------------------------
b'{\n  "error": "rwis_data should contain at least two records with valid t_road."\n}\n'
2012-10-02 17:00:00 fails
-----------------------------


In [9]:
with open('/mnt/HARD/MinMax94/data/JSON/{0}_metro.txt'.format(mm94_station_id), "r") as file:
    z = json.load(file)

In [10]:
z = z['meteo_data']

In [12]:
len(z) / len(mmcc_rwis)

0.9549568443978382